<a href="https://colab.research.google.com/github/huojiajin/NLP-Assignments/blob/main/Week5_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load Libraries
import pandas as pd
import numpy as np

# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

# Read Corpus and convert into a Dataframe
corpusClass = [["money,money,money", "true"],
        ["free money for free gambling fun", "true"],
        ["gambling for fun", "true"],
        ["machine learning for fun,fun,fun", "false"],
        ["free machine learning", "false"]]

df_corpusClass = pd.DataFrame(corpusClass, columns=['Sentence', 'Spam'])
totalDocuments = len(corpusClass)

In [ ]:
# Count Vectorizer
X = cv.fit_transform(df_corpusClass['Sentence'])
vocabulary = len(cv.get_feature_names_out())

# Count Vector
df_ALL = pd.DataFrame(X.toarray(), columns = cv.get_feature_names_out())
df_ALL['Spam'] = df_corpusClass['Spam']
print(df_ALL)

# Compute First Level Probabilities
t = df_ALL.where(df_ALL['Spam'] == 'true').dropna()
p_true = len(t) / totalDocuments

f = df_ALL.where(df_ALL['Spam'] == 'false').dropna()
p_false = len(f) / totalDocuments

print('First Level Probability : True Documents = ', p_true)
print('First Level Probability : False Documents = ', p_false)

   for  free  fun  gambling  learning  machine  money   Spam
0    0     0    0         0         0        0      3   true
1    1     2    1         1         0        0      1   true
2    1     0    1         1         0        0      0   true
3    1     0    3         0         1        1      0  false
4    0     1    0         0         1        1      0  false
First Level Probability : True Documents =  0.6
First Level Probability : False Documents =  0.4


In [ ]:
# Function of Conditional Probabilities
def compute_conditional_probabilities(df):
  # Compute the SUM
  sum_of_columns = df.sum(axis = 0)
  df = df.append(sum_of_columns, ignore_index = True)
  df.at[len(df) - 1, 'Spam'] = 'Sum' 

  # Compute the probabilities
  totalTrueWords = 0
  for i in range(vocabulary):
    totalTrueWords = totalTrueWords + sum_of_columns[i]

  conditional_prob = sum_of_columns

  for i in range(vocabulary):
    conditional_prob[i] = (conditional_prob[i] + 1) / (vocabulary + totalTrueWords)

  df_true_sum_prob = df.append(conditional_prob, ignore_index = True)
  df_true_sum_prob.at[len(df_true_sum_prob) - 1, 'Spam'] = 'Probability'
  return_pd = df_true_sum_prob.T
  
  # Set the last col as the columns' name
  columns = return_pd.loc['Spam']
  return_pd.columns = list(columns)
  return return_pd.drop('Spam')

In [ ]:
# Compute Conditional Probabilities : where spam = true
df_true_sum_prob_T = compute_conditional_probabilities(t)
print("df_true_sum_prob_T:")
print(df_true_sum_prob_T)

# Compute Conditional Probabilities : where spam = false
df_false_sum_prob_T = compute_conditional_probabilities(f)
print("df_false_sum_prob_T:")
print(df_false_sum_prob_T)

df_true_sum_prob_T:
         true true true  Sum Probability
for       0.0  1.0  1.0  2.0    0.157895
free      0.0  2.0  0.0  2.0    0.157895
fun       0.0  1.0  1.0  2.0    0.157895
gambling  0.0  1.0  1.0  2.0    0.157895
learning  0.0  0.0  0.0  0.0    0.052632
machine   0.0  0.0  0.0  0.0    0.052632
money     3.0  1.0  0.0  4.0    0.263158
df_false_sum_prob_T:
         false false  Sum Probability
for        1.0   0.0  1.0       0.125
free       0.0   1.0  1.0       0.125
fun        3.0   0.0  3.0        0.25
gambling   0.0   0.0  0.0      0.0625
learning   1.0   1.0  2.0      0.1875
machine    1.0   1.0  2.0      0.1875
money      0.0   0.0  0.0      0.0625


In [ ]:
test_text = 'machine learning for free';

test_words = test_text.split(" ")

p_true_given_text = 1
p_false_given_text = 1
for i in range(len(test_words)):
  p_true_given_text = p_true_given_text * df_true_sum_prob_T.loc[test_words[i], "Probability"]
  p_false_given_text = p_false_given_text * df_false_sum_prob_T.loc[test_words[i], "Probability"]

print("Probability (spam = true | given text ‘machine learning for free’):",format(p_true_given_text,'.10f'))
print("Probability (spam = false | given text ‘machine learning for free’):",format(p_false_given_text, '.10f'))

Probability (spam = true | given text ‘machine learning for free’): 0.0000690602
Probability (spam = false | given text ‘machine learning for free’): 0.0005493164
